# 실전 문제 해결 (과소적합)

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 100

## 모델 정의

### underfitting되는 모델

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

### DenseNet 사용하여 해결한 모델
- resource를 적게 사용

In [4]:
class MyModel2(tf.keras.Model):
  def __init__(self):
    super(MyModel2, self).__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(32, use_bias = False)
    self.batch1 = tf.keras.layers.BatchNormalization()

    self.dense2 = tf.keras.layers.Dense(32, use_bias = False)
    self.batch2 = tf.keras.layers.BatchNormalization()

    self.dense3 = tf.keras.layers.Dense(64, use_bias = False)
    self.batch3 = tf.keras.layers.BatchNormalization()

    self.dense4 = tf.keras.layers.Dense(128, use_bias = False)
    self.batch4 = tf.keras.layers.BatchNormalization()

    self.dense5 = tf.keras.layers.Dense(10, use_bias = False)

  def call(self, x, training=False, mask=None):
    x = self.flatten(x)

    x = self.dense1(x)
    x = self.batch1(x, training)
    x = tf.nn.relu(x)

    # preactivation
    h = self.batch2(x, training)
    h = tf.nn.relu(h)
    h = self.dense2(h)
    x = tf.concat([x, h], axis=-1)

    h = self.batch3(x, training)
    h = tf.nn.relu(h)
    h = self.dense3(h)
    x = tf.concat([x, h], axis=-1)

    h = self.batch4(x, training)
    h = tf.nn.relu(h)
    h = self.dense4(h)
    x = tf.concat([x, h], axis=-1)

    return self.dense5(x)

## 데이터셋 준비


In [5]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습

In [6]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5167 - accuracy: 0.8118 - val_loss: 0.4140 - val_accuracy: 0.8445
Epoch 2/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3881 - accuracy: 0.8572 - val_loss: 0.4099 - val_accuracy: 0.8498
Epoch 3/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3573 - accuracy: 0.8667 - val_loss: 0.3887 - val_accuracy: 0.8582
Epoch 4/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3345 - accuracy: 0.8741 - val_loss: 0.3844 - val_accuracy: 0.8647
Epoch 5/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3191 - accuracy: 0.8802 - val_loss: 0.3566 - val_accuracy: 0.8726
Epoch 6/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3058 - accuracy: 0.8861 - val_loss: 0.3729 - val_accuracy: 0.8649
Epoch 7/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2965 - accuracy: 0.8898 - val_loss: 0.3518 - val_ac

In [8]:
model2 = MyModel2()
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model2.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
1875/1875 [==============================] - 7s 3ms/step - loss: 2.6704 - accuracy: 0.1799 - val_loss: 2.3038 - val_accuracy: 0.0848
Epoch 2/100
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3035 - accuracy: 0.0871 - val_loss: 2.3026 - val_accuracy: 0.0846
Epoch 3/100
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3026 - accuracy: 0.0913 - val_loss: 2.3026 - val_accuracy: 0.1032
Epoch 4/100
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3027 - accuracy: 0.1290 - val_loss: 2.3026 - val_accuracy: 0.1331
Epoch 5/100
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3026 - accuracy: 0.1320 - val_loss: 2.3026 - val_accuracy: 0.1317
Epoch 6/100
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3026 - accuracy: 0.1324 - val_loss: 2.3026 - val_accuracy: 0.1322
Epoch 7/100
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3026 - accuracy: 0.1319 - val_loss: 2.3026 - val_ac